# 0. Seleção de estratégia de discretização dos *scores* de sentimentos dos comitês de especialistas para comparação com demais algoritmos.

Ao analisar os sentimentos dos algoritmos, os realizamos de forma qualitativa, atribuindo polaridades aos diálogos (negativa, neutra e positiva) e após realizar as rotulações de diálogos, seguindo as instruções da literatura presentes em Kim et al. (2020), determinanos o sentimento final de um diálogo como a média dos sentimentos atribuídos por especialistas (três) humanos componentes de um comitê de avaliação.

Para isso, consideramos os sentimentos sob uma perspectiva quantitativa (numérica), atribuindo valor $-1$ para o sentimento negativo, $0$ para o sentimento neutro e $+1$ para o sentimento positivo, para em sequência, somar as avaliações dos especialistas e dividi-las por três, para calcular a média e usar essa média como sentimento do diálogo (como também para cada expressão contida neste diálogo). Portanto, os sentimentos computados pertencem ao intervalo $\left[-1, 1\right]$

Ao estudar multiplos algoritmos de análise de sentimentos, presentes no *iFeel*, observamos que cada um deles produzem sentimentos em escalas distintas, o que dificulta portanto a comparação com o intervalo que estabelecemos. Felizmente, os autores do *framework* definiram critérios para apontar polaridades dos algoritmos e portanto, torna possível a comparação do desempenho dos algoritmos a partir das polaridades que produzem (negativa, neutra ou positiva) dos sentimentos.

Neste sentido, queremos num primeiro momento, determinar uma estratégia de discretização dos *scores* produzidos pelos comitês para categorizá-los em polaridades e compará-los com os demais algoritmos, de forma que seja possível eleger o algoritmo de análise de sentimentos mais correlacionado aos especialistas humanos, em nível de diálogos (através da análise da força do sentimento de um diálogo --- a partir da agregação de todas suas expressões --- seguindo estratégia proposta por alguns dos autores do *iFeel* com penalização(premiação) fibonacciana dos sentimentos identificados - nossa proposta contribuição) e em nível de expressões, em comparação direta.

Por isso, propomos neste momento, duas estratégias de discretização:

1. Estratégia restritiva:
$$
    \text{polaridade}(\text{sentimento}) = \left\{ {\text{negativo}, \text{sentimento} \lt {-1 \over 3} ; \\ \text{neutro}, {-1 \over 3} \le \text{sentimento} \le {1 \over 3} ; \\ \text{positivo}, \text{sentimento} \gt {1 \over 3} } \right.
$$

1. Estratégia relaxada:
$$
    \text{polaridade}(\text{sentimento}) = \left\{ {\text{negativo}, \text{sentimento} \le {-1 \over 3} ; \\ \text{neutro}, {-1 \over 3} \lt \text{sentimento} \lt {1 \over 3} ; \\ \text{positivo}, \text{sentimento} \ge {1 \over 3} } \right.
$$

Escolheremos a estratégia cuja correlação $\rho$-spearman entre sentimento e qualidade seja maior.

## 0.1 - Carregando diálogos do ConvAI

In [1]:
import pandas as pd
atendimentos_chatbots = pd.read_json('depois.json')

## 0.2 - Aplicando estratégia restritiva para discretização (polarização) dos sentimentos anotados

In [2]:
df_restrito = atendimentos_chatbots.copy()
filtro_negativo_restrito = df_restrito.sentiment < -0.32
inicio_neutro = df_restrito.sentiment >= -0.32
fim_neutro = df_restrito.sentiment <= 0.33
filtro_neutro_restrito = inicio_neutro & fim_neutro
filtro_positivo_restrito = df_restrito.sentiment > 0.33
df_restrito.loc[filtro_negativo_restrito, 'sentiment'] = -1
df_restrito.loc[filtro_neutro_restrito, 'sentiment'] = 0
df_restrito.loc[filtro_positivo_restrito, 'sentiment'] = 1

### 0.2.1 Correlação spearman entre polaridade restrita e qualidade

In [3]:
df_restrito.quality.corr(df_restrito.sentiment, method='spearman')

0.18139246113459112

## 0.3 - Aplicando estratégia relaxada para discretização (polarização) dos sentimentos anotados

In [4]:
df_relaxado = atendimentos_chatbots.copy()
filtro_negativo_relaxado = df_relaxado.sentiment <= -0.32
inicio_neutro = df_relaxado.sentiment > -0.32
fim_neutro = df_relaxado.sentiment < 0.33
filtro_neutro_relaxado = inicio_neutro & fim_neutro
filtro_positivo_relaxado = df_relaxado.sentiment >= 0.33
df_relaxado.loc[filtro_negativo_relaxado, 'sentiment'] = -1
df_relaxado.loc[filtro_neutro_relaxado, 'sentiment'] = 0
df_relaxado.loc[filtro_positivo_relaxado, 'sentiment'] = 1

### 0.3.1 Correlação spearman entre polaridade restrita e qualidade

In [5]:
df_relaxado.quality.corr(df_relaxado.sentiment, method='spearman')

0.18139246113459112

Ao concluir esta comparação verificamos que as estratégias não se diferem, apresentaram a mesma correlação, portanto, vamos aplicar a primeira estratégia para "polarizar" os sentimentos computados pelo comitê

In [6]:
dialogos_sentimentos_polarizados = df_restrito

Com os sentimentos dos diálogos polarizados, agora é hora de submetê-los a análise do ifeel, através de robôs que operam o navegador firefox para fazer upload dos diálogos e baixar o resultado da análise de sentimento deles produzida pelo *iFeel*

# 1. Produção dos arquivos de texto relativos aos diálogos.

Para cada diálogo produziremos três arquivos contendo o diálogo geral (expressões entre humanos e robôs), diálogos dos humanos (apenas expressões ditas pelos humanos) e diálogos dos robôs.

## 1.1 preparativos:

In [22]:
import pdb
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# funcoes auxiliares
compilar_mensagens = lambda x: '\n'.join(list(x))
def gravar_arquivo (caminho, conteudo):
    f = open(caminho, 'w')
    f.write(conteudo)
    f.close()
    
# variaveis auxiliares
login_ifeel = 'dascaniosantos@gmail.com'
senha_ifeel = '123456'

# opcoes do webdriver
options = webdriver.FirefoxOptions()
'''
    descomente a linha abaixo para executar o navegador
    em modo headless
'''
#options.add_argument('--headless')

# criação das pastas de diálogos
!rm -rf dialogos
!mkdir dialogos    

## 1.2 gravacao de dialogos em arquivos txt

In [21]:
# iteração sobre cada dialogo
for index, dialogo in dialogos_sentimentos_polarizados.iterrows():
    mensagens = pd.DataFrame(dialogo.thread)
    if not mensagens.empty:
        mensagens_geral = compilar_mensagens(
            mensagens.text
        )
        gravar_arquivo('dialogos/dialogo_{}.txt'.format(index + 1), mensagens_geral) # para processamento imediato
        mensagens_humano = compilar_mensagens(
            mensagens[mensagens.userId == 'Alice'].text
        )
        gravar_arquivo('dialogos/dialogo_{}_mensagens_humano.txt'.format(index + 1), mensagens_humano) # para processamento posterior
        mensagens_robo = compilar_mensagens(
            mensagens[mensagens.userId == 'Bob'].text
        )
        gravar_arquivo('dialogos/dialogo_{}_mensagens_robo.txt'.format(index + 1), mensagens_robo) # para processamento posterior

## 1.3 Robô de submissão de diálogos para avaliação de resultados dos algoritmos do iFeel

In [24]:
# 1. iniciando o ifeel
!docker start ifeel_container
!docker exec ifeel_container mvn spring-boot:run -Drun.arguments=--spring.profiles.active=docker -Drun.jvmArguments="-Xmx2g" &

ifeel_container


OSError: Background processes not supported.

In [ ]:
# 2. iniciando o robô
driver = webdriver.Firefox(executable_path = '/usr/bin/geckodriver', options = options)